In [39]:
import sys
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
import torch.nn as nn

from skimage.metrics import structural_similarity, peak_signal_noise_ratio, mean_squared_error

plt.rcParams['font.size'] = 14

In [40]:
def calculate_metrics_tf(original_image, encoded_image, message):
    bpp = len(message) * 8 / (original_image.shape[0] * original_image.shape[1])

    im1 = tf.image.convert_image_dtype(original_image, tf.float32)
    im2 = tf.image.convert_image_dtype(encoded_image, tf.float32)
    ssim = float(tf.image.ssim(im1, im2, max_val=1, filter_size=11))

    tensor_img1 = transforms.ToTensor()(original_image)
    tensor_img2 = transforms.ToTensor()(encoded_image)
    mse = nn.functional.mse_loss(tensor_img1, tensor_img2).item()

    psnr = cv2.PSNR(original_image, encoded_image)
    
    return bpp, ssim, mse, psnr

In [41]:
filename = './images/Lenna.png'
filename_encoded = './images/Lenna_steganogan_encoded.png'
message = "sadf"

original_image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
encoded_image = cv2.cvtColor(cv2.imread(filename_encoded), cv2.COLOR_BGR2RGB)

In [42]:
bpp, ssim, mse, psnr = calculate_metrics_tf(original_image, encoded_image, message)
print("BPP:  ", format(float(bpp),  '.10f'))
print("SSIM: ", format(float(ssim), '.10f'))
print("MSE:  ", format(float(mse),  '.10f'))
print("PSNR: ", format(float(psnr), '.10f'))

BPP:   0.0000000000
SSIM:  0.9202553630
MSE:   0.0002659806
PSNR:  35.7515009060


In [66]:
def calculate_metrics(original_image, encoded_image, message):
    bpp = len(message) * 8 / (original_image.shape[0] * original_image.shape[1])
    ssim, _ = structural_similarity(original_image, encoded_image, full=True, channel_axis=-1)   
    mse = mean_squared_error(original_image, encoded_image)
    psnr = peak_signal_noise_ratio(original_image, encoded_image)

    return bpp, ssim, mse, psnr

In [67]:
bpp, ssim, mse, psnr = calculate_metrics_skimage(original_image, encoded_image, message)
print("BPP:  ", format(float(bpp),  '.10f'))
print("SSIM: ", format(float(ssim), '.10f'))
print("MSE:  ", format(float(mse),  '.10f'))
print("PSNR: ", format(float(psnr), '.10f'))

BPP:   0.0000000000
SSIM:  0.9209363268
MSE:   17.2953864455
PSNR:  35.7515009060


In [45]:
def calculate_metrics_func(original_image, encoded_image, message):
    bpp = len(message) * 8 / (original_image.shape[0] * original_image.shape[1])
    ssim, _ = structural_similarity(original_image, encoded_image, full=True, channel_axis=-1)
    mse = np.sum((original_image - encoded_image) ** 2) / float(original_image.size)
    psnr = 10 * np.log10((255 ** 2) / mse)

    return bpp, ssim, mse, psnr

In [46]:
bpp, ssim, mse, psnr = calculate_metrics_func(original_image, encoded_image, message)
print("BPP:  ", format(float(bpp),  '.10f'))
print("SSIM: ", format(float(ssim), '.10f'))
print("MSE:  ", format(float(mse),  '.10f'))
print("PSNR: ", format(float(psnr), '.10f'))

BPP:   0.0000000000
SSIM:  0.9209363268
MSE:   16.0598308899
PSNR:  36.0733939302
